In [1]:
from pywaterinfo import Waterinfo
import os
import pandas as pd
import numpy as np
import schedule
import datetime
import time

In [2]:
vmm_group_list = pd.DataFrame()
hic_group_list = pd.DataFrame()
vmm_group_id = []
hic_group_id = []
vmm_ts_id = pd.DataFrame()
hic_ts_id = pd.DataFrame()
vmm_ts_values = pd.DataFrame()
hic_ts_values = pd.DataFrame()

In [3]:
vmm = Waterinfo("vmm") #flanders enviroment agency
hic = Waterinfo("hic") #flanders hydraulics research

In [4]:
def fetch_vmm_hic_group_list():
    global vmm_group_list,hic_group_list,vmm_group_id,hic_group_id
    vmm_group_list = vmm.get_group_list()
    hic_group_list = hic.get_group_list()
    vmm_group_list = vmm_group_list[(vmm_group_list['group_type'] == 'timeseries') | (vmm_group_list['group_type'] == 'timeseries_dynamic')]
    hic_group_list = hic_group_list[(hic_group_list['group_type'] == 'timeseries') | (hic_group_list['group_type'] == 'timeseries_dynamic')]



    vmm_group_list.to_csv('vmm_group_list.csv',index=False,mode='w')
    hic_group_list.to_csv('hic_group_list.csv',index=False,mode='w')

In [5]:
fetch_vmm_hic_group_list()

In [6]:
def fetch_vmm_hic_group_id():
    global vmm_group_id,hic_group_id,vmm_ts_id,hic_ts_id
    counter = 0;
    vmm_group_list = pd.read_csv('vmm_group_list.csv')
    hic_group_list = pd.read_csv('hic_group_list.csv')

    vmm_group_id = vmm_group_list["group_id"]
    hic_group_id = hic_group_list["group_id"]
    
    for i in vmm_group_id:
        ts_data = vmm.get_timeseries_value_layer(timeseriesgroup_id=i)
        counter=counter+1
        #print(i)
        if ts_data is not None and not ts_data.empty:
            ts_data = ts_data.copy()  # Make a copy to avoid modifying the original DataFrame
            ts_data['group_id'] = i 
            vmm_ts_id = pd.concat([vmm_ts_id,ts_data], ignore_index=True)
        #if counter==10:
            #break
            
            
    vmm_ts_id.to_csv('vmm_ts_id.csv',index=False,header=True,mode='w')

    
    for i in hic_group_id:
        ts_data = hic.get_timeseries_value_layer(timeseriesgroup_id = i)
        if ts_data is not None and not ts_data.empty:
            ts_data = ts_data.copy()  # Make a copy to avoid modifying the original DataFrame
            ts_data['group_id'] = i 
            hic_ts_id = pd.concat([hic_ts_id,ts_data], ignore_index=True)
            
    hic_ts_id.to_csv('hic_ts_id.csv',index=False,header=True,mode='w')

In [7]:
fetch_vmm_hic_group_id()

/tmp/ipykernel_18364/3910304129.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vmm_ts_id = pd.concat([vmm_ts_id,ts_data], ignore_index=True)
/tmp/ipykernel_18364/3910304129.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vmm_ts_id = pd.concat([vmm_ts_id,ts_data], ignore_index=True)
/tmp/ipykernel_18364/3910304129.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA col

In [11]:
def get_vmm_hic_timeseries():
    global vmm_group_id,hic_group_id,vmm_ts_id,hic_ts_id,vmm_ts_values,hic_ts_values
    
    for index, row in vmm_ts_id.iterrows():
        #print(vmm.get_timeseries_values(row['ts_id'], period="P5Y"))
        #vmm_ts_values = pd.concat([vmm_ts_values,vmm.get_timeseries_values(row['ts_id'], period="P1D")], ignore_index=True)
        vmm.get_timeseries_values(row['ts_id'], period="P1D").to_csv('vmm_ts_values.csv',index=False,header=False,mode='a')
        
    #vmm_ts_values.to_csv('vmm_ts_values.csv',index=False,header=True,mode='w')
    
    for index, row in hic_ts_id.iterrows():
        #Whic_ts_values = pd.concat([hic_ts_values,hic.get_timeseries_values(row['ts_id'], period="P1D")], ignore_index=True)
        hic.get_timeseries_values(row['ts_id'], period="P1D").to_csv('hic_ts_values.csv',index=False,header=True,mode='a')
    #hic_ts_values.to_csv('hic_ts_values.csv',index=False,header=True,mode='w')




In [12]:
get_vmm_hic_timeseries()

KiwisException: Waterinfo call returned 429 error with the message b'<?xml version="1.0" ?><ExceptionReport><Exception exceptionCode="AuthCreditLimit"><ExceptionText>Credit limit exceeded for type IP and key 81.82.73.193</ExceptionText></Exception></ExceptionReport>'

In [ ]:
#df = hic.get_timeseries_values(ts_id="39496010", period="P1D")
#print(df)